# Q17. 경쟁적 전염

In [46]:
from collections import deque

n, k = map(int, input().split())

graph = [] # 전체 보드 정보를 담는 리스트
data = [] # 바이러스에 대한 정보를 담는 리스트

for i in range(n):
    #보드 정보를 한 줄 단위로 입력
    graph.append(list(map(int, input().split())))
    for j in range(n):
        #해당 위치에 바이러스가 존재하는 경우
        if graph[i][j] != 0:
            #(바이러스 종류, 시간, 위치X, 위치Y) 삽입
            data.append((graph[i][j], 0, i, j))

#정렬 이후에 큐로 옮기기(낮은 번호의 바이러스가 먼저 증식하므로)
data.sort()
q = deque(data)

target_s, target_x, target_y = map(int, input().split())

#바이러스가 퍼져나갈 수 있는 4가지 위치
dx = [-1, 0 , 1, 0]
dy = [0, 1, 0, -1]

#너비 우선 탐색(BFS) 진행
while q:
    virus, s, x, y = q.popleft()
    # 정확히 s초가 지나거나, 큐가 빌 때까지 반복
    if s == target_s:
        break
    # 현재 노드에서 주변 4가지 위치를 각각 확인
    for i in range(4):
        nx = x+dx[i]
        ny = y+dy[i]
        #해당 위치로 이동할 수 있는 경우
        if 0 <= nx and nx < n and 0 <= ny and ny < n:
            #아직 방문하지 않은 위치라면, 그 위치에 바이러스 넣기
            if graph[nx][ny] == 0:
                graph[nx][ny] = virus
                q.append((virus, s+1, nx, ny))

print(graph[target_x-1][target_y-1])

3 3
1 0 2
0 0 0
3 0 0
2 3 2
3


# Q18. 괄호 변환

In [9]:
# 균형잡힌 괄호 문자열의 인덱스 반환
def balanced_index(p):
    count = 0 
    for i in range(len(p)):
        if p[i] =='(':
            count += 1
        else:
            count -= 1
        if count ==0:
            return i

#올바른 괄호 문자열인지 판단
def check_proper(p):
    count = 0 #왼쪽 괄호의 개수
    for i in p:
        if i == '(':
            count += 1
        else:
            if count == 0: #쌍이 맞지 않는 경우에 False 반환
                return False
            count -= 1
        return True #쌍이 맞는 경우에 True 반환

def solution(p):
    answer =''
    if p =='':
        return answer
    index = balanced_index(p)
    u = p[:index+1]
    v = p[index+1:]
    #올바른 괄호 문자열이면, v에 대해 함수를 수행한 결과를 붙여 반환
    if check_proper(u):
        answer = u+solution(v)
    #올바른 괄호 문자열이 아니라면 아래의 과정을 수행
    else:
        answer = '('
        answer += solution(v)
        answer += ')'
        u = list(u[1:-1]) #첫 번째와 마지막 문자를 제거
        for i in range(len(u)):
            if u[i] == '(':
                u[i] = ')'
            else:
                u[i] = '('
        answer += "".join(u)
    return answer


In [10]:
p = '(()))('
solution(p)

'(())()'

# Q19. 연산자 끼워 넣기

- 중복 순열 이용 (중복 가능한 n개에서 r개를 택하여 일렬로 나열하는 경우의 수.)

In [15]:
n = int(input())
data = list(map(int, input().split()))
add, sub, mul, div = map(int, input().split())

min_value = 1e9
max_value = -1e9

#깊이 우선 탐색(DFS) 메서드
def dfs(i, now):
    global min_value, max_value, add, sub, mul, div
    #모든 연산자를 다 사용한 경우, 최솟값과 최댓값 업데이트
    if i == n:
        min_value = min(min_value, now)
        max_value = max(max_value, now)
    else:
        #각 연산자에 대하여 재귀적으로 수행
        if add > 0 :
            add -= 1
            dfs(i+1, now+data[i])
            add += 1 # 여러 조합을 찾아봐야하니까 원래대로 되돌릴려고 
        if sub > 0:
            sub -= 1
            dfs(i+1, now-data[i])
            sub += 1
        if mul > 0:
            mul -= 1
            dfs(i+1, now*data[i])
            mul += 1
        if div > 0:
            div -= 1
            dfs(i+1, int(now/data[i])) #나눌 때는 나머지를 제거
            div += 1
#DFS 메서드 호출
dfs(1, data[0])

print(max_value)
print(min_value)

2
5 6
0 0 1 0
30
30


아래는 경진언니 코드

In [1]:
from itertools import permutations

n = int(input()) # 숫자 개수
array = list(map(int, input().split())) # 수열
count = list(map(int, input().split())) # +-*/ 각 개수

# +-*/를 1234로 개수만큼 리스트(oper)에 추가
oper = []
for i in range(4):
    oper += [i] * count[i]

# 가능한 연산 순서 조합 계산(중복 제거)
oper = list(set(permutations(oper, len(oper))))

# 연산 순서가 주어졌을 때 연산 결과 계산
def calculate(op):
    result = array[0]
    for i in range(n-1):
        if op[i] == 0:
            result += array[i+1]
        elif op[i] == 1:
            result -= array[i+1]
        elif op[i] == 2:
            result *= array[i+1]
        else:
            # 둘 중 하나가 음수일 경우
            if result * array[i+1] < 0 :
                result = -(abs(result) // abs(array[i+1]))
            # 그 외
            else:
                result //= array[i+1]
    return result

# 첫 번째 연산 조합에 대한 계산 결과를 초기값으로 설정
res_max = calculate(oper[0])
res_min = calculate(oper[0])

# 각 연산 조합에 대해 결과 계산
for i in range(1,len(oper)):
    res_max = max(res_max, calculate(oper[i]))
    res_min = min(res_min, calculate(oper[i]))

# 결과 출력
print(res_max)
print(res_min)

2
5 6
0 0 1 0
30
30
